COMBINAR

In [ ]:
import os
import open3d as o3d

# Directorio donde están los archivos .pcd
dir_path = "/scans/"

# Obtener una lista de todos los archivos .pcd en el directorio
pcd_files = [f for f in os.listdir(dir_path) if f.endswith('.pcd')]

# Leer el primer archivo .pcd
combined_pcd = o3d.io.read_point_cloud(os.path.join(dir_path, pcd_files[0]))

# Leer y unir el resto de los archivos .pcd
for file in pcd_files[1:]:
    pcd = o3d.io.read_point_cloud(os.path.join(dir_path, file))
    combined_pcd += pcd

# Guardar el archivo .pcd combinado
o3d.io.write_point_cloud("mapa.pcd", combined_pcd)

VOXELIZAR

In [ ]:
import os
import numpy as np
import open3d as o3d

def voxel_filter(pcd, voxel_size):
    try:
        downpcd = pcd.voxel_down_sample(voxel_size=voxel_size)
        return downpcd
    except Exception as e:
        print(f"Error during voxel filtering: {e}")
        return None

def process_segment(pcd, bounds, voxel_size):
    try:
        # Filtrar puntos dentro de estos límites
        segment_pcd = pcd.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=bounds[0], max_bound=bounds[1]))

        # Aplicar filtro de voxel al segmento
        filtered_segment = voxel_filter(segment_pcd, voxel_size)
        return filtered_segment
    except Exception as e:
        print(f"Error during segment processing: {e}")
        return None

def main():
    pcd_file_path = "mapa.pcd"
    
    voxel_size = 0.1

    if not os.path.exists(pcd_file_path):
        print(f"PCD file not found: {pcd_file_path}")
        return

    try:
        print("Loading PCD file...")
        # Cargar archivo PCD
        pcd = o3d.io.read_point_cloud(pcd_file_path)

        # Verificar si la nube de puntos está vacía
        if not pcd.has_points():
            print("The point cloud is empty.")
            return

        print("Counting initial points...")
        # Contar número de puntos antes del filtrado
        initial_point_count = np.asarray(pcd.points).shape[0]
        print(f"Initial point count: {initial_point_count}")

        # Extraer los límites mínimos y máximos de la nube de puntos
        min_bound = pcd.get_min_bound()
        max_bound = pcd.get_max_bound()

        # Definir el número de divisiones en cada eje (x, y, z)
        divisions = (2, 2, 2)  # Ajusta según el tamaño y la capacidad de memoria disponible
        step = (max_bound - min_bound) / divisions

        all_filtered_points = []

        # Dividir la nube de puntos en segmentos y procesar cada uno
        for i in range(divisions[0]):
            for j in range(divisions[1]):
                for k in range(divisions[2]):
                    segment_min = min_bound + step * [i, j, k]
                    segment_max = segment_min + step
                    print(f"Processing segment: min={segment_min}, max={segment_max}")
                    filtered_segment = process_segment(pcd, (segment_min, segment_max), voxel_size)
                    if filtered_segment is not None:
                        all_filtered_points.append(filtered_segment.points)

        # Combinar todos los puntos filtrados en una nueva nube de puntos
        filtered_pcd = o3d.geometry.PointCloud()
        for segment_points in all_filtered_points:
            filtered_pcd += o3d.geometry.PointCloud(o3d.utility.Vector3dVector(segment_points))

        print("Counting final points...")
        # Contar número de puntos después del filtrado
        final_point_count = np.asarray(filtered_pcd.points).shape[0]
        print(f"Final point count: {final_point_count}")

        # Guardar el archivo voxelizado resultante
        output_file = pcd_file_path.replace(".pcd", "_voxelizado.pcd")
        print(f"Saving filtered point cloud to {output_file}...")
        o3d.io.write_point_cloud(output_file, filtered_pcd)
        
        print(f"Número de puntos antes del filtrado: {initial_point_count}")
        print(f"Número de puntos después del filtrado: {final_point_count}")
        print(f"Puntos filtrados guardados en {output_file}")

    except Exception as e:
        print(f"Error en el voxelizaco: {e}")

if __name__ == "__main__":
    main()


FILTRAR

In [8]:
import open3d as o3d
import numpy as np

def filter_z_using_sigma(pcd_file_path, sigma_threshold=3):
    # Cargar el archivo PCD
    pcd = o3d.io.read_point_cloud(pcd_file_path)
    
    # Extraer las coordenadas z
    z_coords = np.asarray(pcd.points)[:, 2]  # Las coordenadas z están en la tercera columna
    
    # Calcular la media y la desviación estándar de las coordenadas z
    z_mean = np.mean(z_coords)
    z_std = np.std(z_coords)
    
    # Calcular los límites para filtrar usando sigma
    lower_bound = z_mean - sigma_threshold * z_std
    upper_bound = z_mean + sigma_threshold * z_std
    
    # Crear un filtro basado en los límites
    filter_mask = (z_coords >= lower_bound) & (z_coords <= upper_bound)
    
    # Aplicar el filtro para mantener solo los puntos dentro de los límites
    filtered_points = pcd.select_by_index(np.where(filter_mask)[0])
    
    # Crear un nuevo archivo PCD para guardar los puntos filtrados
    output_file_path = pcd_file_path.replace(".pcd", "_filtrado.pcd")
    o3d.io.write_point_cloud(output_file_path, filtered_points)
    print(f"Filtered {len(filtered_points.points)} points within {sigma_threshold} sigma(s).")

if __name__ == '__main__':
    pcd_file_path = "mapa_voxelizado.pcd"
    filter_z_using_sigma(pcd_file_path, sigma_threshold=7)

Filtered 100972 points within 7 sigma(s).


CONVERTIR A .LAS

In [ ]:
### CONVERSIÓN A .LAS
import subprocess

def convert_pcd_to_las(pcd_file_path, las_file_path):
    # Comando PDAL para convertir PCD a LAS
    command = f"pdal translate {pcd_file_path} {las_file_path}"

    try:
        # Ejecutar el comando
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Mostrar la salida del comando
        print(result.stdout.decode())
        print(f"Archivo LAS guardado en {las_file_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error al ejecutar el comando: {e.stderr.decode()}")

def main():
    pcd_file_path = "mapa_voxelizado_filtrado.pcd"
    las_file_path = pcd_file_path.replace(".pcd", ".las")
    
    convert_pcd_to_las(pcd_file_path, las_file_path)

if __name__ == "__main__":
    main()


GEORREFERENCIAR

In [ ]:
import subprocess
import pyproj
import laspy
from scipy.spatial.transform import Rotation as R
import numpy as np
import os

class PointCloudTransformer:
    def __init__(self, initial_gps_coords, initial_orientation, latest_path_coords):
        self.initial_gps_coords = initial_gps_coords
        self.initial_orientation = initial_orientation
        self.latest_path_coords = latest_path_coords

    def transform_point_cloud(self, file_path, rotation_angle_deg, displacement_xyz):
        if self.initial_gps_coords is not None and self.initial_orientation is not None and self.latest_path_coords is not None:
            # Leer el archivo .las convertido
            las_data = laspy.read(file_path)

            # Obtener las coordenadas de los puntos en la nube de puntos
            points = np.vstack((las_data.x, las_data.y, las_data.z)).T

            # Restar información de la trayectoria de la nube de puntos
            #points[:, 0] -= self.latest_path_coords[0]
            #points[:, 1] -= self.latest_path_coords[1]
            #points[:, 2] -= self.latest_path_coords[2]

            # Aplicar la rotación en el plano XY
            rotation_angle_rad = np.radians(rotation_angle_deg)
            rotation_matrix = np.array([[np.cos(rotation_angle_rad), -np.sin(rotation_angle_rad), 0],
                                        [np.sin(rotation_angle_rad), np.cos(rotation_angle_rad), 0],
                                        [0, 0, 1]])
            points = np.dot(rotation_matrix, points.T).T
            print("Rotación aplicada")

            # Desplazar la nube de puntos según los valores proporcionados
            points[:, 0] += displacement_xyz[0]
            points[:, 1] += displacement_xyz[1]
            points[:, 2] += displacement_xyz[2]
            print("Desplazamiento aplicado")

            # Agregar coordenadas GPS para georreferenciar la nube de puntos
            points[:, 0] += self.initial_gps_coords[0]
            points[:, 1] += self.initial_gps_coords[1]
            points[:, 2] += self.initial_gps_coords[2]  

            # Actualizar las coordenadas de los puntos en la nube de puntos
            las_data.x = points[:, 0].astype(np.float64)
            las_data.y = points[:, 1].astype(np.float64)
            las_data.z = points[:, 2].astype(np.float64)

            # Guardar la nube de puntos transformada en un nuevo archivo .las
            new_file_path = file_path.replace(".las", "georreferenciado.las")
            las_data.write(new_file_path)
            print(f"Nube de puntos transformada y guardada en: {new_file_path}")
        else:
            print("Esperando información completa de GPS, IMU y trayectoria para realizar la transformación.")

def main():
    # Valores de transformación  
    rotation_angle_deg = 0# Ángulo de rotación en grados
    displacement_xyz = [368027.3 , 4064258.1, 38]  # Desplazamiento en las direcciones X, Y, Z

    # Valores de ejemplo para la inicialización del transformador
    initial_gps_coords = (0, 0, 0)  # Coordenadas GPS iniciales
    initial_orientation = R.from_euler('xyz', [0, 0, 0], degrees=True)  # Orientación inicial 
    latest_path_coords = (0, 0, 0)  # Coordenadas de la trayectoria más reciente

    # Crear una instancia del transformador de nube de puntos
    transformer = PointCloudTransformer(initial_gps_coords, initial_orientation, latest_path_coords)

    # Ruta al archivo .las
    las_file_path = "mapa_voxelizado_filtrado.las"

    # Realizar las transformaciones en el archivo .las
    transformer.transform_point_cloud(las_file_path, rotation_angle_deg, displacement_xyz)

if __name__ == "__main__":
    main()

